# Using Historical Data to Predict Batting Success: Step 2 PLAY graphs

Authored by: Donna J. Harris (994042890)

Email: harr2890@mylaurier.ca

For: CP640 Machine Learning (S22) with Professor Elham Harirpoush

## Step 2 of Data Preparation   TODO

Step 2 ... means we've done the first part that is common to all work, and now we're ready to experiment specifically from this point with other data formats...

TODO....

## Environment Setup

Import and establish environment for initial work, including showing all dataframe column values.

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)

Load in the two stored data files: `./data/core_mlb_dataset.csv` and `./data/hof_dataset.csv` so we can continue with pre-processing the data.


In [2]:
core_mlb_dataset = "./data/core_mlb_dataset.csv"
df = pd.read_csv(core_mlb_dataset)
df

,ID,Player,Tm,Opp,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF,Result,Season
0,delahed01,Ed Delahanty,PHI,BRO,5,4,1,2,0,0,0,0,1,0,0,0,0,L,1901
1,dolanjo02,Joe Dolan,PHI,BRO,5,5,0,1,0,0,0,1,0,0,0,0,0,L,1901
2,childcu01,Cupid Childs,CHC,STL,5,5,1,1,0,0,0,0,0,0,0,0,0,W,1901
3,crolifr01,Fred Crolius,BSN,NYG,4,4,0,0,0,0,0,1,0,0,0,0,0,W,1901
4,delahed01,Ed Delahanty,PHI,BRO,4,4,0,0,0,0,0,0,0,2,0,0,0,L,1901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715517,woodfja01,Jake Woodford,STL,CHC,2,1,0,0,0,0,0,0,0,0,0,1,0,L,2021
3715518,yastrmi01,Mike Yastrzemski,SFG,SDP,4,3,1,1,1,0,0,2,1,1,0,0,0,W,2021
3715519,zimmebr01,Bradley Zimmer,CLE,TEX,4,4,1,2,0,0,0,1,0,0,0,0,0,W,2021
3715520,zimmery01,Ryan Zimmerman,WSN,BOS,4,3,0,0,0,0,0,1,1,2,0,0,0,L,2021


In [3]:
hof_dataset = "./data/hof_dataset.csv"
hof = pd.read_csv(hof_dataset)
hof

,ID,Inductee
0,hodgegi01,1
1,kaatji01,1
2,minosmi01,1
3,olivato01,1
4,ortizda01,1
...,...,...
263,cobbty01,1
264,johnswa01,1
265,mathech01,1
266,ruthba01,1


## Preprocessing - Continued

### Looking at Player Data

So far, we've looked at data from the perspective of each individual game played by an individual player. Now the data is healthier than when we started, we want to look at individual players during their careers.

Keeping in mind that our goal is to find trends that might predict a batter's success in the future, we will want to be able to combine their statistics over time. It might also be useful to attempt to find ways to examine their data during parts of their careers, but that's not currently a focus

In [4]:
print("There are",len(pd.unique(df['ID'])),"unique players remaining in this dataset.")

There are 14336 unique players remaining in this dataset.


In [5]:
columns = ['ID', 'Player','PA','AB','R','H','2B','3B','HR','RBI','BB','SO','HBP','SH', 'SF']
summables = ['PA','AB','R','H','2B','3B','HR','RBI','BB','SO','HBP','SH', 'SF']

career = df.copy()
career = career[columns].copy()
career

,ID,Player,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF
0,delahed01,Ed Delahanty,5,4,1,2,0,0,0,0,1,0,0,0,0
1,dolanjo02,Joe Dolan,5,5,0,1,0,0,0,1,0,0,0,0,0
2,childcu01,Cupid Childs,5,5,1,1,0,0,0,0,0,0,0,0,0
3,crolifr01,Fred Crolius,4,4,0,0,0,0,0,1,0,0,0,0,0
4,delahed01,Ed Delahanty,4,4,0,0,0,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3715517,woodfja01,Jake Woodford,2,1,0,0,0,0,0,0,0,0,0,1,0
3715518,yastrmi01,Mike Yastrzemski,4,3,1,1,1,0,0,2,1,1,0,0,0
3715519,zimmebr01,Bradley Zimmer,4,4,1,2,0,0,0,1,0,0,0,0,0
3715520,zimmery01,Ryan Zimmerman,4,3,0,0,0,0,0,1,1,2,0,0,0


In [6]:
career = career.groupby(['ID', 'Player'])
career = career[summables].sum().copy()
career

,,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF
ID,Player,,,,,,,,,,,,,
aardsda01,David Aardsma,5,4,0,0,0,0,0,0,0,2,0,1,0
aaronha01,Henry Aaron,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120
aaronto01,Tommie Aaron,1045,944,99,216,42,6,13,94,85,145,0,9,6
aasedo01,Don Aase,5,5,0,0,0,0,0,0,0,3,0,0,0
abadan01,Andy Abad,25,21,1,2,0,0,0,0,4,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuninmi01,Mike Zunino,2835,2559,308,518,111,5,141,345,198,981,58,8,12
zupcibo01,Bob Zupcic,886,795,93,199,47,4,7,80,57,137,6,20,8
zupofr01,Frank Zupo,8,7,1,2,1,0,0,0,1,2,0,0,0


**Note:** Now we've created our career statistics records for every player, we want to remove any records with 0 At Bats. We know that these players will not be in the Hall of Fame and it will avoid problems with calculating other statistics later.

In [7]:
career = career[career['AB'] > 0].copy()
career

,,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF
ID,Player,,,,,,,,,,,,,
aardsda01,David Aardsma,5,4,0,0,0,0,0,0,0,2,0,1,0
aaronha01,Henry Aaron,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120
aaronto01,Tommie Aaron,1045,944,99,216,42,6,13,94,85,145,0,9,6
aasedo01,Don Aase,5,5,0,0,0,0,0,0,0,3,0,0,0
abadan01,Andy Abad,25,21,1,2,0,0,0,0,4,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuninmi01,Mike Zunino,2835,2559,308,518,111,5,141,345,198,981,58,8,12
zupcibo01,Bob Zupcic,886,795,93,199,47,4,7,80,57,137,6,20,8
zupofr01,Frank Zupo,8,7,1,2,1,0,0,0,1,2,0,0,0


### Different Statistical Formulae

singles (int) = H - (2B + 3B + HR)

total_bases (int) = 1 * singles + 2 * 2B + 3 * 3B + 4 * HR


**AVG (float) = H / AB**

SLG (float) = total_bases / AB

OBP (float) = (H + BB + HBP)/(AB + BB + HBP + SF)

**OPS (float) = SLG + OBP**
https://en.wikipedia.org/wiki/On-base_plus_slugging  *** Reference is great for testing against

RC (float) = total_bases * ( (H + BB) / (AB + BB) )

ISO (float) = (1 * 2B + 2 * 3B + 3 * HR) / AB

PA/SO (float) = PA/SO

#### Simplification of SLG formula??

SLG = ( 1 * (H - (2B + 3B + HR)) + 2 * 2B + 3 * 3B + 4 * HR ) / AB
( H - 2B - 3B - HR + 2 * 2B + 3 * 3B + 4 * HR ) / AB
( H - 2B + 2 * 2B - 3B + 3 * 3B - HR + 4 * HR ) / AB
( H + 2B + 2 * 3B + 3 * HR ) / AB

In [8]:
career['AVG'] = career['H'] / (career['AB']*1.0)

career['SLG'] = (career['H'] + career['2B'] + 2*career['3B'] + 3*career['HR']) / (career['AB']*1.0)

career['OBP'] = (career['H'] + career['BB'] + career['HBP']) / ((career['AB'] + career['BB'] + career['HBP'] + career['SF'])*1.0) 

career['OPS'] = career['SLG'] + career['OBP']

career

,,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF,AVG,SLG,OBP,OPS
ID,Player,,,,,,,,,,,,,,,,,
aardsda01,David Aardsma,5,4,0,0,0,0,0,0,0,2,0,1,0,0.000000,0.000000,0.000000,0.000000
aaronha01,Henry Aaron,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120,0.305503,0.555152,0.374276,0.929429
aaronto01,Tommie Aaron,1045,944,99,216,42,6,13,94,85,145,0,9,6,0.228814,0.327331,0.290821,0.618152
aasedo01,Don Aase,5,5,0,0,0,0,0,0,0,3,0,0,0,0.000000,0.000000,0.000000,0.000000
abadan01,Andy Abad,25,21,1,2,0,0,0,0,4,5,0,0,0,0.095238,0.095238,0.240000,0.335238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zuninmi01,Mike Zunino,2835,2559,308,518,111,5,141,345,198,981,58,8,12,0.202423,0.415006,0.273788,0.688794
zupcibo01,Bob Zupcic,886,795,93,199,47,4,7,80,57,137,6,20,8,0.250314,0.345912,0.302540,0.648452
zupofr01,Frank Zupo,8,7,1,2,1,0,0,0,1,2,0,0,0,0.285714,0.428571,0.375000,0.803571


### Merging with Hall of Fame Data

In [9]:
career = pd.merge(hof, career, on="ID", how="right")
career

,ID,Inductee,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF,AVG,SLG,OBP,OPS
0,aardsda01,NaN,5,4,0,0,0,0,0,0,0,2,0,1,0,0.000000,0.000000,0.000000,0.000000
1,aaronha01,1.0,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120,0.305503,0.555152,0.374276,0.929429
2,aaronto01,NaN,1045,944,99,216,42,6,13,94,85,145,0,9,6,0.228814,0.327331,0.290821,0.618152
3,aasedo01,NaN,5,5,0,0,0,0,0,0,0,3,0,0,0,0.000000,0.000000,0.000000,0.000000
4,abadan01,NaN,25,21,1,2,0,0,0,0,4,5,0,0,0,0.095238,0.095238,0.240000,0.335238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14259,zuninmi01,NaN,2835,2559,308,518,111,5,141,345,198,981,58,8,12,0.202423,0.415006,0.273788,0.688794
14260,zupcibo01,NaN,886,795,93,199,47,4,7,80,57,137,6,20,8,0.250314,0.345912,0.302540,0.648452
14261,zupofr01,NaN,8,7,1,2,1,0,0,0,1,2,0,0,0,0.285714,0.428571,0.375000,0.803571
14262,zuvelpa01,NaN,545,491,40,109,17,2,2,20,34,50,2,18,0,0.221996,0.276986,0.275142,0.552128


In [10]:
hof_is_nan = career.loc[pd.isna(career['Inductee'])]
series_of_hof = hof_is_nan.index
career.loc[series_of_hof, 'Inductee'] = 0
career['Inductee'] = career['Inductee'].astype(int)
career

,ID,Inductee,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF,AVG,SLG,OBP,OPS
0,aardsda01,0,5,4,0,0,0,0,0,0,0,2,0,1,0,0.000000,0.000000,0.000000,0.000000
1,aaronha01,1,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120,0.305503,0.555152,0.374276,0.929429
2,aaronto01,0,1045,944,99,216,42,6,13,94,85,145,0,9,6,0.228814,0.327331,0.290821,0.618152
3,aasedo01,0,5,5,0,0,0,0,0,0,0,3,0,0,0,0.000000,0.000000,0.000000,0.000000
4,abadan01,0,25,21,1,2,0,0,0,0,4,5,0,0,0,0.095238,0.095238,0.240000,0.335238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14259,zuninmi01,0,2835,2559,308,518,111,5,141,345,198,981,58,8,12,0.202423,0.415006,0.273788,0.688794
14260,zupcibo01,0,886,795,93,199,47,4,7,80,57,137,6,20,8,0.250314,0.345912,0.302540,0.648452
14261,zupofr01,0,8,7,1,2,1,0,0,0,1,2,0,0,0,0.285714,0.428571,0.375000,0.803571
14262,zuvelpa01,0,545,491,40,109,17,2,2,20,34,50,2,18,0,0.221996,0.276986,0.275142,0.552128


In [11]:
name_cols = ['ID', 'Player']
names = df.copy()
names = names[name_cols].copy()
names = names.drop_duplicates(subset=['ID', 'Player'], keep='first')
names

,ID,Player
0,delahed01,Ed Delahanty
1,dolanjo02,Joe Dolan
2,childcu01,Cupid Childs
3,crolifr01,Fred Crolius
5,demonge01,Gene DeMontreville
...,...,...
3715012,houckta01,Tanner Houck
3715063,minteaj01,A.J. Minter
3715191,adonjo01,Joan Adon
3715410,paynety01,Tyler Payne


In [12]:
career = pd.merge(names, career, on="ID", how="right")
career

,ID,Player,Inductee,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,HBP,SH,SF,AVG,SLG,OBP,OPS
0,aardsda01,David Aardsma,0,5,4,0,0,0,0,0,0,0,2,0,1,0,0.000000,0.000000,0.000000,0.000000
1,aaronha01,Henry Aaron,1,13666,12121,2128,3703,614,96,740,2243,1372,1357,32,21,120,0.305503,0.555152,0.374276,0.929429
2,aaronto01,Tommie Aaron,0,1045,944,99,216,42,6,13,94,85,145,0,9,6,0.228814,0.327331,0.290821,0.618152
3,aasedo01,Don Aase,0,5,5,0,0,0,0,0,0,0,3,0,0,0,0.000000,0.000000,0.000000,0.000000
4,abadan01,Andy Abad,0,25,21,1,2,0,0,0,0,4,5,0,0,0,0.095238,0.095238,0.240000,0.335238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14259,zuninmi01,Mike Zunino,0,2835,2559,308,518,111,5,141,345,198,981,58,8,12,0.202423,0.415006,0.273788,0.688794
14260,zupcibo01,Bob Zupcic,0,886,795,93,199,47,4,7,80,57,137,6,20,8,0.250314,0.345912,0.302540,0.648452
14261,zupofr01,Frank Zupo,0,8,7,1,2,1,0,0,0,1,2,0,0,0,0.285714,0.428571,0.375000,0.803571
14262,zuvelpa01,Paul Zuvella,0,545,491,40,109,17,2,2,20,34,50,2,18,0,0.221996,0.276986,0.275142,0.552128


In [13]:
career = career[['ID', 'Player', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'AVG', 'SLG', 'OBP', 'OPS', 'Inductee']]
career

,ID,Player,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,AVG,SLG,OBP,OPS,Inductee
0,aardsda01,David Aardsma,5,4,0,0,0,0,0,0,0,2,0.000000,0.000000,0.000000,0.000000,0
1,aaronha01,Henry Aaron,13666,12121,2128,3703,614,96,740,2243,1372,1357,0.305503,0.555152,0.374276,0.929429,1
2,aaronto01,Tommie Aaron,1045,944,99,216,42,6,13,94,85,145,0.228814,0.327331,0.290821,0.618152,0
3,aasedo01,Don Aase,5,5,0,0,0,0,0,0,0,3,0.000000,0.000000,0.000000,0.000000,0
4,abadan01,Andy Abad,25,21,1,2,0,0,0,0,4,5,0.095238,0.095238,0.240000,0.335238,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14259,zuninmi01,Mike Zunino,2835,2559,308,518,111,5,141,345,198,981,0.202423,0.415006,0.273788,0.688794,0
14260,zupcibo01,Bob Zupcic,886,795,93,199,47,4,7,80,57,137,0.250314,0.345912,0.302540,0.648452,0
14261,zupofr01,Frank Zupo,8,7,1,2,1,0,0,0,1,2,0.285714,0.428571,0.375000,0.803571,0
14262,zuvelpa01,Paul Zuvella,545,491,40,109,17,2,2,20,34,50,0.221996,0.276986,0.275142,0.552128,0


In [14]:
career[career['Inductee'] == 1 ]

,ID,Player,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,AVG,SLG,OBP,OPS,Inductee
1,aaronha01,Henry Aaron,13666,12121,2128,3703,614,96,740,2243,1372,1357,0.305503,0.555152,0.374276,0.929429,1
197,alomaro01,Roberto Alomar,10400,9073,1507,2724,504,80,210,1134,1032,1140,0.300231,0.442852,0.371245,0.814097,1
325,aparilu01,Luis Aparicio,10972,10003,1294,2610,383,91,80,770,713,718,0.260922,0.341398,0.309698,0.651095,1
331,applilu01,Luke Appling,10254,8856,1317,2750,441,101,45,1110,1313,531,0.310524,0.398374,0.400196,0.798570,1
395,ashburi01,Richie Ashburn,9442,8103,1286,2492,305,106,29,577,1169,553,0.307540,0.382081,0.396700,0.778780,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13903,wilsoha01,Hack Wilson,5555,4760,885,1460,266,67,244,1063,672,714,0.306723,0.544538,0.394718,0.939255,1
13949,winfida01,Dave Winfield,12358,11003,1669,3110,540,88,465,1833,1216,1686,0.282650,0.474507,0.352622,0.827129,1
14107,wynnea01,Early Wynn,1869,1675,135,361,59,5,17,174,138,323,0.215522,0.287164,0.275426,0.562590,1
14128,yastrca01,Carl Yastrzemski,13992,11988,1816,3419,646,59,452,1844,1845,1395,0.285202,0.462045,0.379453,0.841499,1


In [15]:
career.isnull().values.any()


False

In [16]:
hank_row = career[career['ID'] == 'aaronha01']

cols = ['PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'AVG', 'SLG', 'OBP', 'OPS']
XX = hank_row[cols]

XX


,PA,AB,R,H,2B,3B,HR,RBI,BB,SO,AVG,SLG,OBP,OPS
1,13666,12121,2128,3703,614,96,740,2243,1372,1357,0.305503,0.555152,0.374276,0.929429


### Converting to X and y

In [17]:
y = career[career.columns[-1]]
y = y.values
y

array([0, 1, 0, ..., 0, 0, 0])

In [18]:
# remove the HOF inductee column, and the string-based fields
num = career.shape[1]
X = career[career.columns[2:num-1]]
X = X.values
X

# X = career[career.columns[2:num-5]]
# X = X.values
# X

array([[5.00000000e+00, 4.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.36660000e+04, 1.21210000e+04, 2.12800000e+03, ...,
        5.55152215e-01, 3.74276292e-01, 9.29428507e-01],
       [1.04500000e+03, 9.44000000e+02, 9.90000000e+01, ...,
        3.27330508e-01, 2.90821256e-01, 6.18151765e-01],
       ...,
       [8.00000000e+00, 7.00000000e+00, 1.00000000e+00, ...,
        4.28571429e-01, 3.75000000e-01, 8.03571429e-01],
       [5.45000000e+02, 4.91000000e+02, 4.00000000e+01, ...,
        2.76985743e-01, 2.75142315e-01, 5.52128058e-01],
       [1.56000000e+02, 1.33000000e+02, 4.00000000e+00, ...,
        1.72932331e-01, 1.85714286e-01, 3.58646617e-01]])

### Visualization(s)

??? Before and after bar graphs?

## Model Selection

### Logistic Regression

In [19]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

# Split data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

# Import logistic regression 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Build a logstic regression object
LogReg = LogisticRegression(solver = 'newton-cg')

# Train(fit) model
LogReg.fit(X_train, y_train)

# Predict the data
y_train_predict = LogReg.predict(X_train)
y_test_predict = LogReg.predict(X_test)

# Count percentage of correct predictions
print("The performance of the model")
print("--------------------------------------")
print('Log loss of the model for training set: %.3f' % log_loss(y_train,y_train_predict))
print('Log loss of the model for test set: %.3f' % log_loss(y_test,y_test_predict))

The performance of the model
--------------------------------------
Log loss of the model for training set: 0.315
Log loss of the model for test set: 0.351


In [20]:
# Count percentage of correct predictions
print("The performance of the model:")
print("------------------------------")

# Confusion Matrix
from sklearn.metrics import confusion_matrix
print('Confusion matrix for training set')
print(confusion_matrix(y_train,y_train_predict))

print('Confusion matrix for test set')
print(confusion_matrix(y_test, y_test_predict),'\n')

# Accuracy Score
from sklearn.metrics import accuracy_score
print('Accuracy of the model for training set: %.3f' % accuracy_score(y_train,y_train_predict))
print('Accuracy of the model for test set: %.3f\n' % accuracy_score(y_test,y_test_predict))

# Precision score
from sklearn.metrics import precision_score
print('Precision of the model for training set: %.3f' % precision_score(y_train,y_train_predict))
print('Precision of the model for test set: %.3f\n' % precision_score(y_test,y_test_predict))

# Recall score
from sklearn.metrics import recall_score
print('Recall of the model for training set: %.3f' % recall_score(y_train,y_train_predict))
print('Recall of the model for test set: %.3f\n' % recall_score(y_test,y_test_predict))

# F1 Score
from sklearn.metrics import f1_score
print('F1-measure of the model for training set: %.3f' % f1_score(y_train,y_train_predict))
print('F1-measure of the model for test set: %.3f\n' % f1_score(y_test,y_test_predict))

# AUC score
from sklearn.metrics import roc_auc_score
print('AUC score of the model for training set: %.3f' % roc_auc_score(y_train,y_train_predict))
print('AUC score of the model for test set: %.3f\n' % roc_auc_score(y_test,y_test_predict))
      
# Log loss
print('Log loss of the model for training set: %.3f' % log_loss(y_train,y_train_predict))
print('Log loss of the model for test set: %.3f\n' % log_loss(y_test,y_test_predict))


The performance of the model:
------------------------------
Confusion matrix for training set
[[11257    12]
 [   92    50]]
Confusion matrix for test set
[[2816    3]
 [  26    8]] 

Accuracy of the model for training set: 0.991
Accuracy of the model for test set: 0.990

Precision of the model for training set: 0.806
Precision of the model for test set: 0.727

Recall of the model for training set: 0.352
Recall of the model for test set: 0.235

F1-measure of the model for training set: 0.490
F1-measure of the model for test set: 0.356

AUC score of the model for training set: 0.676
AUC score of the model for test set: 0.617

Log loss of the model for training set: 0.315
Log loss of the model for test set: 0.351



In [25]:
pip install scikit-plot

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [23]:
import matplotlib.pyplot as plt

In [24]:
import scikitplot as skplt

### 